# Linear transformations over data matrix

---
> Erick Eduardo Aguilar Hernández:
> * mat.ErickAguilar@gmail.com.mx
> * isc.ErickAguilar@gmail.com

---

Let $V$, $W$ be a vector spaces over $\mathbb{R}$  with $dim(V)=p$, $dim(W)=m$. Let $T: V \subset \mathbb{R}^p$ $\rightarrow$ $W \subset \mathbb{R}^m$ be a linear transformation. Then there exists a unique $p \times m$ matrix $\mathbf{A}$ such that $T\left[v\right]=A\left[v\right]$ for every $v \in V$. 

In some types of analysis a linear transformation of the data matrix it's required, so that each observation should be transformed to another vector space, note that each row could be interpreted as a vector in $\mathbb{R}^p$, so the matrix expression associated to the linear transformation have be expressed as:

$$
\begin{align*} 
\mathbf{Y}_{m\times n}  & = \mathbf{A}_{m\times p}[\mathbf{X}]'_{p\times n}  \\
\mathbf{Y'}_{n\times m} & = \mathbf{X}_{n\times p}[\mathbf{A}]'_{p\times m} \\
\mathbf{Z}_{n\times m}  & = \mathbf{X}_{n\times p}[\mathbf{B}]_{p\times m} \\
\end{align*}
$$

This transformation produces a new data matrix $\textbf{Z}$.

$$
\left[ \begin{array}{ccccc}
z_{1 1} & \cdots & z_{1 m} \\
\vdots  & \ddots & \vdots \\
z_{i 1} & \cdots & z_{i m} \\
\vdots  & \ddots & \vdots \\
z_{n 1} & \cdots & z_{n m} \end{array} \right]
= \left[ \begin{array}{ccccc}
x_{1 1} & \cdots & x_{1 j} & \cdots & x_{1 p} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
x_{i 1} & \cdots & x_{i j} & \cdots & x_{i p} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
x_{n 1} & \cdots & x_{n j} & \cdots & x_{n p} \end{array} \right]
\left[ \begin{array}{ccccc}
b_{1 1} & \cdots & b_{1 m} \\
\vdots  & \ddots & \vdots \\
b_{j 1} & \cdots & b_{i m} \\
\vdots  & \ddots & \vdots \\
b_{p 1} & \cdots & b_{p m} \end{array} \right]
$$

Where the k-th column could be expressed in terms of linear combinations:

$$
\begin{align*}
z_{1 k}   & = b_{1 k} x_{1 1} + \cdots b_{j k} x_{1 j} + \cdots + b_{p k} x_{1 p}   \\
\vdots    & = \vdots \\
z_{n k}   & = b_{1 k} x_{1 1} + \cdots b_{j k} x_{1 j} + \cdots + b_{p k} x_{1 p} \\
\end{align*}
$$

Properties:
*  $\mathbf{\bar{y}}=\frac{1}{n}[\textbf{1}_{1\times n}]\textbf{Y} = \frac{1}{n}[\textbf{1}_{1\times n}]\textbf{XA} = \mathbf{\bar{x}A}$
   
*  $\mathbf{S_y}=\frac{1}{n}\textbf{Y'HY}=\frac{1}{n}\textbf{[XA]'H[XA]}=\frac{1}{n}\textbf{A'X'HXA}=\textbf{A'}\frac{1}{n}\textbf{X'HX}\textbf{A}=\textbf{A'SA}$

Consider the centered data matrix $\textbf{Y} = \textbf{HX}$

#### Proyection matrix
---

If $V$ it's a subspace of $\mathbb{R}^p$ whose basis is $\{\mathbf{\alpha}_1,\dotsc,\mathbf{\alpha}_k\}$ then $\forall \mathbf{v} \in V$ with $x_i=1,\dotsc, k \in \mathbb{R}$


$$
\begin{align*} 
\mathbf{v} & = x_1 \mathbf{\alpha}_1+\dotsc+x_k \mathbf{\alpha}_k \\
 & = x_1
\left[ \begin{array}{c}
\alpha_{1 1}\\
\vdots \\
\alpha_{p 1}\end{array} \right]
+\dotsc+x_k
\left[ \begin{array}{c}
\alpha_{1 k}\\
\vdots \\
\alpha_{p k}\end{array} \right] \\
& = \left[ \begin{array}{ccccc}
x_1 \alpha_{1 1} + & \cdots & + x_k \alpha_{1 k} \\
\vdots  & \ddots & \vdots  \\
x_1 \alpha_{p 1} + & \cdots & + x_k \alpha_{p k} \end{array} \right]\\
& = \left[ \begin{array}{ccccc}
\alpha_{1 1} & \cdots & \alpha_{1 k} \\
\vdots  & \ddots & \vdots  \\
\alpha_{p 1} & \cdots & \alpha_{p k} \end{array} \right]
\left[ \begin{array}{ccccc}
x_1 \\
\vdots \\
x_k \end{array} \right]\\
\mathbf{v} &= \mathbf{A}_{p \times k} \mathbf{x}_{k \times 1}
\end{align*}
$$

if $\textbf{y} \in \mathbb{R}^p $ then $Proy_V\textbf{y} \in V$ and $Proy_V\textbf{y}=\mathbf{A} \mathbf{x}$ for some $\textbf{x} \in \mathbb{R}^k $ also $\textbf{y} = Proy_V\textbf{y}+Proy_{V^{\perp}}\textbf{y}$ where $V^{\perp}= Span\left(\{\mathbf{\alpha}_1,\dotsc,\mathbf{\alpha}_k\}\right)^{\perp}=Kernel(A')$ then

$$
\begin{align*}
Proy_{V^{\perp}}\textbf{y} & = \textbf{y} - Proy_V\textbf{y} \in Kernel(A')\\
\implies \textbf{A'}\left(\textbf{y} - Proy_V\textbf{y}\right) & = \textbf{0}\\
\textbf{A'}\textbf{y} - \textbf{A'} Proy_V\textbf{y} & = \textbf{0}\\
\textbf{A'}\textbf{y} - \textbf{A'} \mathbf{A}\mathbf{x} & = \textbf{0}\\
\textbf{A'}\textbf{y} & = \textbf{A'}\mathbf{A}\mathbf{x} \\
\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\mathbf{A}\mathbf{x} \\
\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \mathbf{A}^{-1}[\textbf{A'}]^{-1}\textbf{A'}\mathbf{A}\mathbf{x} \\
\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \mathbf{I}\mathbf{x}\\
\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \mathbf{x} \\
\mathbf{A}\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = \mathbf{Ax}\\
\mathbf{A}\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y} & = Proy_V\textbf{y}\\
\end{align*}
$$

The matrix $\mathbf{A}\left(\textbf{A'}\mathbf{A}\right)^{-1}\textbf{A'}\textbf{y}$ are called the proyection matrix. 

**Observation:**

The centering matrix $\textbf{H}$ it's a projection matrix of the rows $\textbf{x}_i$ of the data matrix, onto the **(n-1)** dimensional subspace that is orthogonal to the nullspace $\mathbf{1}$ (This is the subspace of all n-vectors whose components sum to zero).

**Example 1.2 - 1 [Computing linear transformations using Spark UDF's]**

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Column as c
from pyspark.sql.functions import array, udf, lit, col as c
import numpy as np
try:
    sc = SparkContext('local[*]')
except:
    sc = SparkContext.getOrCreate('local[*]')
sqlContext = SQLContext(sc)

In [2]:
gradesPath = '../DataSets/Grades.txt'
gradesDF = sqlContext.read.format('com.databricks.spark.csv')\
                   .options(header='true',inferschema='true')\
                   .load(gradesPath)
gradesDF.toPandas()

,Student,Phisics,Maths,French,History,Literature
0,1,7,7,5,5,6
1,2,5,5,6,6,5
2,3,5,6,5,7,5
3,4,6,8,5,6,6
4,5,7,6,6,7,6
5,6,4,4,6,7,6
6,7,5,5,5,5,6
7,8,5,6,5,5,5
8,9,6,5,7,6,6
9,10,6,5,6,6,6


In [3]:
def toLiteralArray(arguments):
    return array(*[lit(value) for value in arguments])

innerProduct = udf(lambda v,w : float(np.inner(v,w)))
cols = array(c('Phisics'),c('Maths'),c('French'),c('History'),c('Literature'))
ColVector1 = toLiteralArray([1/5,1/5,1/5,1/5,1/5])
ColVector2 = toLiteralArray([1/2,1/2,0,0,0])
ColVector3 = toLiteralArray([0,0,1/3,1/3,1/3])

In [4]:
averagesDF = gradesDF.select(innerProduct(cols,ColVector1).alias('AverageGrade'),\
                             innerProduct(cols,ColVector2).alias('Sciences'),\
                             innerProduct(cols,ColVector3).alias('Letters'))
averagesDF.toPandas()

,AverageGrade,Sciences,Letters
0,6.000000000000001,7.0,5.333333333333333
1,5.4,5.0,5.666666666666666
2,5.6000000000000005,5.5,5.666666666666666
3,6.2,7.0,5.666666666666666
4,6.400000000000001,6.5,6.333333333333333
5,5.4,4.0,6.333333333333333
6,5.2,5.0,5.333333333333333
7,5.2,5.5,5.0
8,6.000000000000001,5.5,6.333333333333333
9,5.800000000000001,5.5,6.0


#### Standarized data matrix
---

The standardized data matrix is defined as that whose values $x_{ij}$ are standardized per column, i.e. $z_{ij}=\frac{x_{ij}-\bar{x}_j}{\sqrt{S_{jj}}}$, this matrix can be written as a product of linear transformations

$$
\begin{align*}
\textbf{Z}_{n \times p} & = \left[ \begin{array}{ccccc}
\frac{x_{11}-\bar{x}_{1}}{\sqrt{S_{11}}} & \cdots &  \frac{x_{i1}-\bar{x}_{1}}{\sqrt{S_{jj}}} & \cdots & \frac{x_{n1}-\bar{x}_{1}}{\sqrt{S_{pp}}}\\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
\frac{x_{1j}-\bar{x}_{2}}{\sqrt{S_{11}}} & \cdots & \frac{x_{ij}-\bar{x}_{2}}{\sqrt{S_{jj}}} & \cdots & \frac{x_{nj}-\bar{x}_{2}}{\sqrt{S_{pp}}} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
\frac{x_{1p}-\bar{x}_{p}}{\sqrt{S_{11}}} & \cdots & \frac{x_{ip}-\bar{x}_{p}}{\sqrt{S_{jj}}} & \cdots & \frac{x_{np}-\bar{x}_{p}}{\sqrt{S_{pp}}} 
\end{array} \right] \\
& = \left[ \begin{array}{ccccc}
x_{11}-\bar{x}_{1} & \cdots & x_{1j}-\bar{x}_{2} & \cdots & x_{1p}-\bar{x}_{p} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
x_{i1}-\bar{x}_{1} & \cdots & x_{ij}-\bar{x}_{2} & \cdots & x_{ip}-\bar{x}_{p} \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
x_{n1}-\bar{x}_{1} & \cdots & x_{nj}-\bar{x}_{2} & \cdots & x_{np}-\bar{x}_{p} 
\end{array} \right]
\left[ \begin{array}{ccccc}
\frac{1}{\sqrt{S_{11}}} & \cdots &0 & \cdots & 0 \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
0 & \cdots & \frac{1}{\sqrt{S_{jj}}} & \cdots & 0 \\
\vdots  & \ddots & \vdots  & \ddots & \vdots \\
0 & \cdots & 0 & \cdots &\frac{1}{\sqrt{S_{pp}}}
\end{array} \right] \\ \\
\textbf{Z}_{n \times p} & = \textbf{H X D}^{-1/2} \\ 
\end{align*}
$$